In [2]:
import random
import time
import tracemalloc
from tqdm import tqdm
import pandas as pd

In [3]:
def dfs_recursive(graph, start, visited=None):
    if visited is None:
        visited = set()

    visited.add(start)

    for neighbor in graph[start]:
        if neighbor not in visited:
            dfs_recursive(graph, neighbor, visited)

    return visited

## Временная сложность

Для ориентированного/неориентированного графа:

- каждый узел посещается один раз — `O(V)`

- каждое ребро рассматривается один раз — `O(E)`

Итого: `O(V+E)`

## Пространственная сложность

- рекурсивный стек: глубина до `V`

- множество посещённых: `O(V)`

Итого: `O(V)`

In [4]:
def dfs_iterative(graph, start):
    visited = set()
    stack = [start]

    while stack:
        node = stack.pop()
        if node not in visited:
            visited.add(node)
            stack.extend(graph[node])

    return visited


## Алгоритмическая оптимизация

Для DFS реальной алгоритмической оптимизации нет, так как асимптотика оптимальна `O(V+E)`.

Но возможны улучшения:

- Убрать рекурси, стек вручную (итеративный DFS)

Избавляемся от overhead рекурсии и ограничений глубины стека.

Преимущества:

- устойчив к глубине графа

- быстрее, т.к. нет рекурсивных вызовов

In [5]:
def generate_graph(n, edges_per_node=5):
    graph = {i: [] for i in range(n)}
    for i in range(n):
        for _ in range(edges_per_node):
            graph[i].append(random.randint(0, n-1))
    return graph


In [6]:
def benchmark_recursive_n(graph, n_runs=1000):
    import tracemalloc
    import time

    times = []
    peaks = []

    for i in range(n_runs):
        tracemalloc.start()
        t0 = time.time()

        try:
            dfs_recursive(graph, 0)
        except RecursionError as e:
            tracemalloc.stop()
            continue

        t1 = time.time()
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        run_time = t1 - t0
        times.append(run_time)
        peaks.append(peak)

    if len(times) == 0:
        return {'avg_time': None, 'avg_peak_mem': None}

    return {'avg_time': sum(times)/len(times), 'avg_peak_mem': sum(peaks)/len(peaks)/1024}


In [7]:
def benchmark_iterative_n(graph, n_runs=1000):
    import tracemalloc
    import time

    times = []
    peaks = []

    for i in range(n_runs):
        tracemalloc.start()
        t0 = time.time()

        dfs_iterative(graph, 0)

        t1 = time.time()
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        run_time = t1 - t0
        times.append(run_time)
        peaks.append(peak)

    if not times:
        return {'avg_time': None, 'avg_peak_mem': None}

    return {'avg_time': sum(times)/len(times), 'avg_peak_mem': sum(peaks)/len(peaks)/1024}


In [ ]:
n_values = [500, 1_000, 10_000]
results = []

for n in tqdm(n_values):
    graph = generate_graph(n)

    rec = benchmark_recursive_n(graph, n_runs=1000)
    it = benchmark_iterative_n(graph, n_runs=1000)

    results.append({
        'graph_size': n,
        'recursive_avg_time': rec['avg_time'],
        'recursive_avg_peak_mem_KB': rec['avg_peak_mem'],
        'iterative_avg_time': it['avg_time'],
        'iterative_avg_peak_mem_KB': it['avg_peak_mem']
    })

100%|██████████| 3/3 [00:54<00:00, 18.27s/it]


In [17]:
df = pd.DataFrame(results)
df

,n_runs,recursive_avg_time,recursive_avg_peak_mem_KB,iterative_avg_time,iterative_avg_peak_mem_KB
0,500,0.002218,54.294973,0.000298,48.513328
1,1000,0.008197,66.263979,0.000540,50.920258
2,10000,NaN,NaN,0.018640,794.484406
